In [ ]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
import time

# Tworzenie klasy


In [ ]:
class WebScrapper:

    def __init__(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        self.browser = webdriver.Chrome('chromedriver', options=chrome_options)
        
        self.browser.get("https://www.gry-online.pl/gry/12vi")
        time.sleep(1)

        doc = self.browser.find_elements_by_xpath("//button[contains(@class,'css-1xrfbkv')]")
        if len(doc):
            doc[0].click()
            time.sleep(1)

    def next_page(self, how_many=1):
        for i in range(how_many):
            doc = self.browser.find_elements_by_xpath("//div[contains(@class,'np-right')]")[0]
            doc.click()
            time.sleep(1)

    def extract_games(self):
        gry_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        return gry_soup.find_all('h5')

    def extract_links_part(self):
        gry_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        return gry_soup.find_all('a', class_="pic-c")

    def extract_short_text(self):
        gry_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
        return gry_soup.find_all('p', class_=False)
    
    def extract_desc(self, addres):
        return BeautifulSoup(requests.get('https://www.gry-online.pl' + addres).text, 'html.parser')

    def close(self):
        self.browser.quit()



# Pobieranie i obrabianie daych

In [ ]:
getter = WebScrapper()
df = pd.DataFrame({'Game_name': getter.extract_games(), 'html': getter.extract_links_part(), 'Short_text': getter.extract_short_text()[2:]})

repetition = 1
for i in range(repetition):
    if i%10 == 0:
        print(np.round(i/repetition)*100, ' %')
    getter.next_page()
    df = df.append(pd.DataFrame({'Game_name': getter.extract_games(), 'html': getter.extract_links_part(), 'Short_text': getter.extract_short_text()[2:]}), ignore_index = True)

df['Game_name'] = df['Game_name'].apply(lambda x: x.get_text())
df['Short_text'] = df['Short_text'].apply(lambda x: x.get_text())
df['html'] = df['html'].apply(lambda x: x['href'])



In [ ]:
def test_computing(x): 
    x = BeautifulSoup(x)
    res = x.find_all(id="game-description-cnt")
    d = str(res[0]).find('>Dodatkowe')
    kt = str(res[0]).find('>Kwestie')
    inne = str(res[0]).find('>Inne')
    lista = [d, kt, inne]
    if np.min(lista) == -1:
        if np.max(lista) == -1:
            return(BeautifulSoup(str(res[0])).get_text())
        else:
            min_ = min(list(map(lambda x: np.Inf if x == -1 else x, lista)))
            return(BeautifulSoup(str(res[0])[0: min_]).get_text())
    else:
        return(BeautifulSoup(str(res[0])[0: np.min(lista)]).get_text())

In [ ]:
df['Opis'] = df['html'].apply(getter.extract_desc)
getter.close()
df['Opis'] = df['Opis'].apply(lambda x: test_computing(x))
df.drop_duplicates(subset=['Game_name'],inplace=True)
df.dropna(axis=0,inplace=True)
df = df.reset_index(drop=True)
df.info()

In [ ]:
df.to_csv('list_v7_final.csv')